<a href="https://colab.research.google.com/github/khadeom/ML/blob/main/Genre%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python_speech_features

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for python_speech_features: filename=python_speech_features-0.6-py3-none-any.whl size=5886 sha256=b2230571e160bed16a065d9ca847a3e76c9cdd20ee67033db103ee496ce4ff2f
  Stored in directory: /root/.cache/pip/wheels/09/a1/04/08e2688d2562d8f9ff89e77c6ddfbf7268e07dae1a6f22455e
Successfully built python_speech_features


In [ ]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile

import os
import pickle
import random
import operator

import math

In [ ]:
# function to get the distance between feature vecotrs and find neighbors
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [ ]:
# identify the class of the instance
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)

    return sorter[0][0]

In [ ]:
# function to evaluate the model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    
    return (1.0 * correct) / len(testSet)

In [ ]:
# ! pip install -q kaggle

In [ ]:
# # get data
# from google.colab import files
# files.upload()





In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
 ! chmod 600 ~/.kaggle/kaggle.json
 

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
# directory that holds the wav files
directory = r"/content/drive/MyDrive/Colab Notebooks/Data/genres_original/"

# binary file where we will collect all the features extracted using mfcc (Mel Frequency Cepstral Coefficients)
f = open("my.dat", 'wb')

i = 0

for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):        
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)        

f.close()

Got an exception:  File format b'\xcb\x15\x1e\x16' not understood. Only 'RIFF' and 'RIFX' supported.  in folder:  jazz  filename:  jazz.00054.wav


In [ ]:
# Split the dataset into training and testing sets respectively
dataset = []

def loadDataset(filename, split, trSet, teSet):
    with open('my.dat', 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])
trainingSet = []
testSet = []
loadDataset('my.dat', 0.66, trainingSet, testSet)

In [ ]:
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [ ]:
# making predictions using KNN
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

0.6869565217391305


In [ ]:
# # testing the code with external samples
# test_dir = r"/content/drive/MyDrive/Colab Notebooks/Test/"
# test_file = test_dir + "test1.wav"
# test_file = test_dir + "test2.wav"
# test_file = test_dir + "test3.wav"
# test_file = test_dir + "test4.wav"
# test_file = test_dir + "test5.wav"

In [ ]:
from collections import defaultdict
results = defaultdict(int)

directory = r"/content/drive/MyDrive/Colab Notebooks/Data/genres_original/"
i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
def getNeighbors(trainingSet, instance, k):
    distances = []
    # print("instance",instance)
    # print(trainingSet)
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))

    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

In [ ]:
def feature_generation(test_file):

  (rate, sig) = wav.read(test_file)
  mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
  covariance = np.cov(np.matrix.transpose(mfcc_feat))
  mean_matrix = mfcc_feat.mean(0)
  feature = (mean_matrix, covariance, i)
  return feature

In [ ]:
# feature_generation("/content/drive/MyDrive/Colab Notebooks/Test/test2.wav")

In [ ]:
!apt install fluidsynth
!wget https://github.com/FluidSynth/fluid-soundfont/releases/download/v3.1/FluidR3_GM.sf2 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
Suggested packages:
  fluid-soundfont-gs timidity jackd musescore
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth2 libinstpatch-1.0-2 qsynth
  timgm6mb-soundfont
0 upgraded, 6 newly installed, 0 to remove and 24 not upgraded.
Need to get 126 MB of archives.
After this operation, 157 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 libinstpatch-1.0-2 amd64 1.1.2-2build1 [238 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 timgm6mb-soundfont all 1.3-3 [5,420 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libfluidsynth2 amd64 2.1.1-2 [

In [ ]:
import subprocess

def midi_to_wav(midi_filename, wav_filename):
    """
    Convert a MIDI file to a WAV file using FluidSynth.
    """
    subprocess.call(['fluidsynth', '-F', wav_filename, '-ni', 'FluidR3_GM.sf2', midi_filename])


midi_to_wav("/content/generatedAudio7.midi", "/content/wav_file.wav")

In [ ]:
# "/content/drive/MyDrive/Colab Notebooks/Test/test2.wav"
test_dir = r"/content/drive/MyDrive/Colab Notebooks/Test/"
test_file = test_dir + "test1.wav"
for file in os.listdir(test_dir):
  print(file)
  a=test_dir+file
  if file.split('.')[-1] == 'midi':
    main_file =test_dir+ file.split(".")[0] + ".wav"
    midi_to_wav(a,main_file)
    pred = nearestClass(getNeighbors(dataset, feature_generation(main_file), 5))
    print (results[pred])

test2.wav
test1.wav
test4.wav
test3.wav
test5.wav
output.wav
example.midi


jazz
GeneratedAudio0.midi


classical
GeneratedAudio1.midi


classical
GeneratedAudio2.midi


classical
GeneratedAudio3.midi


...

classical
GeneratedAudio46.midi


classical
GeneratedAudio47.midi


classical
GeneratedAudio48.midi


classical
GeneratedAudio49.midi


classical
output.mid


In [ ]:
# "/content/drive/MyDrive/Colab Notebooks/Test/test2.wav"
test_dir = r"/content/drive/MyDrive/Colab Notebooks/New_wave/"
test_file = test_dir + "wav_file.wav"
for file in os.listdir(test_dir):
  print(file)
  pred = nearestClass(getNeighbors(dataset, feature_generation(test_dir+file), 5))
  print (results[pred])

wav_file.wav


classical


## wav to midi

In [ ]:
# Install FluidSynth library and soundfont
!apt-get install fluidsynth
!wget "https://freepats.zenvoid.org/Piano/acoustic_grand_piano_ydp_20080910.sf2"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (2.1.1-2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2023-04-16 14:51:20--  https://freepats.zenvoid.org/Piano/acoustic_grand_piano_ydp_20080910.sf2
Resolving freepats.zenvoid.org (freepats.zenvoid.org)... 173.236.185.252
Connecting to freepats.zenvoid.org (freepats.zenvoid.org)|173.236.185.252|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2023-04-16 14:51:20 ERROR 404: Not Found.



In [ ]:
pip install pyFluidSynth

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#not a part of genre classification
import fluidsynth
import numpy as np
import wave

def wav_to_mid(file_name):

  # Load the soundfont into FluidSynth
  sf2 = "acoustic_grand_piano_ydp_20080910.sf2"
  fs = fluidsynth.Synth()
  sfid = fs.sfload(sf2)

  # Load the WAV file into a numpy array
  wav_file = wave.open(file_name, "rb")
  nframes = wav_file.getnframes()
  audio = np.frombuffer(wav_file.readframes(nframes), dtype=np.int16)

  # Convert the audio to MIDI using FluidSynth
  fs.start()
  fs.program_select(0, sfid, 0, 0)

  for i in range(0, len(audio), 1024):
      samples = audio[i:i+1024].astype(float) / 32768.0
      fs.write(np.int16(fs.float2pcm(fs.midi_noteon(0, 60, 127))))
      for sample in samples:
          fs.write(np.int16(fs.float2pcm(fs.midi_pitch_bend(0, int(sample*8192) + 8192))))
      fs.write(np.int16(fs.float2pcm(fs.midi_noteoff(0, 60, 127))))

  fs.delete()

  # Save the resulting MIDI file
  with open(file_name.split(".")+".midi", "wb") as midi_file:
      fs.midi_file_write("/content/drive/MyDrive/Colab Notebooks/Genre_classification/Data/gener_midi/"+midi_file)
